In [18]:
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
#Answer to 1
jan = pd.read_csv('./clean-january.csv')
# A little bit of a hack to use the dataframes' apply method
# (which is faster than iteration)
# and obtian the number of short trips.
count = 0
def dealx(x):
    if x < 3:
        global count
        count += 1
        return 1
    else:
        return 0
    
# This is a pretty efficient way of doing both creating
# a new pandas dataframe, inserting the required value
# and getting a count for the number of short trips.
jan['short_trip'] = jan['trip_distance'].apply(lambda x: dealx(x))

# These are what we want to access to store in a 
# the new dataframe.
cols = ['short_trip', 'trip_distance', 'passenger_count', 'payment_amount', 'duration']
new_cols = ['Short_Trip', 'Trip_distance', 'Passenger_count', 'Payment_amount', 'Duration']

new_jan = pd.DataFrame()
new_jan['Index'] = jan.index.tolist()
new_jan[new_cols] = jan[cols]

print('Question 1')
print('% of short trips={0:.1f}'.format((count/(new_jan['Short_Trip'].shape[0]))*100))
print('*'*3)

Question 1
% of short trips=71.3
***


In [20]:
# Answer to question 2
print('Question 2 median trip distance')
print('January:', jan['trip_distance'].median())
print('*'*3)
print('Question 2 median trip duration')
print('January', jan['duration'].median())
print('*'*3)

Question 2 median trip distance
January: 1.82
***
Question 2 median trip duration
January 11.0
***


In [21]:
# Answer to question 3
# The amount of normal values is the sum of the boolean values
# of the series between 1.2 and 4.5
normal_count = jan['trip_distance'].between(1.2, 4.5).sum()
total_count = jan['trip_distance'].shape[0]
print('Question 3')
# The abnormal count is the total count
# less the normal count.
print("{0:.1f}% of instances in ‘tripdistance’ are abnormal! (normal range is between 1.2 and 4.5)"
      .format(100*(total_count-normal_count)/(total_count)))
print('*'*3)

Question 3
42.9% of instances in ‘tripdistance’ are abnormal! (normal range is between 1.2 and 4.5)
***


In [22]:
# Answer to question 4
jan['pickup_datetime'] = pd.to_datetime(jan['pickup_datetime'], format='%d/%m/%y %H:%M')

In [23]:
vals

0       2016-01-01 00:00:00
1       2016-01-01 00:01:00
2       2016-01-01 00:02:00
3       2016-01-01 00:03:00
4       2016-01-01 00:04:00
5       2016-01-01 00:04:00
6       2016-01-01 00:05:00
7       2016-01-01 00:05:00
8       2016-01-01 00:06:00
9       2016-01-01 00:06:00
10      2016-01-01 00:06:00
11      2016-01-01 00:07:00
12      2016-01-01 00:08:00
13      2016-01-01 00:08:00
14      2016-01-01 00:09:00
15      2016-01-01 00:09:00
16      2016-01-01 00:09:00
17      2016-01-01 00:09:00
18      2016-01-01 00:10:00
19      2016-01-01 00:10:00
20      2016-01-01 00:10:00
21      2016-01-01 00:11:00
22      2016-01-01 00:11:00
23      2016-01-01 00:11:00
24      2016-01-01 00:11:00
25      2016-01-01 00:11:00
26      2016-01-01 00:12:00
27      2016-01-01 00:12:00
28      2016-01-01 00:12:00
29      2016-01-01 00:13:00
                ...        
89214   2016-01-31 23:34:00
89215   2016-01-31 23:34:00
89216   2016-01-31 23:35:00
89217   2016-01-31 23:35:00
89218   2016-01-31 2

In [17]:
vals[:,]

0       2016-01-01 00:00:00
1       2016-01-01 00:01:00
2       2016-01-01 00:02:00
3       2016-01-01 00:03:00
4       2016-01-01 00:04:00
5       2016-01-01 00:04:00
6       2016-01-01 00:05:00
7       2016-01-01 00:05:00
8       2016-01-01 00:06:00
9       2016-01-01 00:06:00
10      2016-01-01 00:06:00
11      2016-01-01 00:07:00
12      2016-01-01 00:08:00
13      2016-01-01 00:08:00
14      2016-01-01 00:09:00
15      2016-01-01 00:09:00
16      2016-01-01 00:09:00
17      2016-01-01 00:09:00
18      2016-01-01 00:10:00
19      2016-01-01 00:10:00
20      2016-01-01 00:10:00
21      2016-01-01 00:11:00
22      2016-01-01 00:11:00
23      2016-01-01 00:11:00
24      2016-01-01 00:11:00
25      2016-01-01 00:11:00
26      2016-01-01 00:12:00
27      2016-01-01 00:12:00
28      2016-01-01 00:12:00
29      2016-01-01 00:13:00
                ...        
89214   2016-01-31 23:34:00
89215   2016-01-31 23:34:00
89216   2016-01-31 23:35:00
89217   2016-01-31 23:35:00
89218   2016-01-31 2